In [ ]:
import pandas as pd
import os
from pathlib import Path
import glob

# Đường dẫn đến thư mục Dataset
dataset_path = Path('Dataset')
en_path = dataset_path / 'en'
es_path = dataset_path / 'es'

# Định nghĩa các category
categories = ['News', 'Others', 'Reviews', 'Social Media']

print("="*80)
print("ĐỌC DỮ LIỆU TỪ CÁC FOLDER VÀ CATEGORY")
print("="*80)

# Danh sách để lưu trữ tất cả dataframes
all_dfs = []

# Đọc dữ liệu từ tiếng Anh
print("\n🇬🇧 Tiếng Anh (EN):")
for category in categories:
    category_path = en_path / category
    if category_path.exists():
        files = glob.glob(str(category_path / '*.tsv'))
        print(f"\n  📁 Category: {category}")
        print(f"     Tìm thấy {len(files)} files")
        
        for file in files:
            try:
                df = pd.read_csv(file, sep='\t', header=None, names=['sentiment', 'text', 'score'])
                df['language'] = 'en'
                df['category'] = category
                df['source'] = os.path.basename(file).replace('.tsv', '')
                all_dfs.append(df)
                print(f"     ✓ {os.path.basename(file)}: {len(df):,} dòng")
            except Exception as e:
                print(f"     ✗ Lỗi khi đọc {file}: {e}")

# Đọc dữ liệu từ tiếng Tây Ban Nha
print("\n🇪🇸 Tiếng Tây Ban Nha (ES):")
for category in categories:
    category_path = es_path / category
    if category_path.exists():
        files = glob.glob(str(category_path / '*.tsv'))
        print(f"\n  📁 Category: {category}")
        print(f"     Tìm thấy {len(files)} files")
        
        for file in files:
            try:
                df = pd.read_csv(file, sep='\t', header=None, names=['sentiment', 'text', 'score'])
                df['language'] = 'es'
                df['category'] = category
                df['source'] = os.path.basename(file).replace('.tsv', '')
                all_dfs.append(df)
                print(f"     ✓ {os.path.basename(file)}: {len(df):,} dòng")
            except Exception as e:
                print(f"     ✗ Lỗi khi đọc {file}: {e}")

# Gộp tất cả dữ liệu
combined_df = pd.concat(all_dfs, ignore_index=True)

print("\n" + "="*80)
print("THỐNG KÊ DỮ LIỆU BAN ĐẦU")
print("="*80)
print(f"Tổng số dòng: {len(combined_df):,}")

print(f"\n📊 Phân bố theo Ngôn ngữ:")
for lang, count in combined_df['language'].value_counts().items():
    lang_name = 'English' if lang == 'en' else 'Spanish'
    print(f"   {lang_name} ({lang}): {count:,}")

print(f"\n📊 Phân bố theo Category:")
for cat, count in combined_df['category'].value_counts().items():
    print(f"   {cat}: {count:,}")

print(f"\n📊 Phân bố theo Sentiment:")
for sent, count in combined_df['sentiment'].value_counts().sort_index().items():
    sent_name = 'Negative' if sent == -1 else ('Neutral' if sent == 0 else 'Positive')
    print(f"   {sent_name} ({sent}): {count:,}")

# Hiển thị phân bố chi tiết
print(f"\n📊 Phân bố Language x Category:")
print(pd.crosstab(combined_df['language'], combined_df['category']))

print(f"\n📊 Phân bố Language x Sentiment:")
print(pd.crosstab(combined_df['language'], combined_df['sentiment']))

print(f"\n📊 Phân bố Category x Sentiment:")
print(pd.crosstab(combined_df['category'], combined_df['sentiment']))

# ==================================================
# CÂN BẰNG DỮ LIỆU THEO 3 CHIỀU
# ==================================================
print("\n" + "="*80)
print("CÂN BẰNG DỮ LIỆU THEO NGÔN NGỮ, CATEGORY VÀ SENTIMENT")
print("="*80)

# Tính số lượng mẫu cho mỗi tổ hợp (language, category, sentiment)
group_counts = combined_df.groupby(['language', 'category', 'sentiment']).size()
print("\nSố lượng mẫu cho mỗi tổ hợp (Language x Category x Sentiment):")
print(group_counts.sort_index())

# Tìm số lượng nhỏ nhất
min_samples_per_group = group_counts.min()
print(f"\n🎯 Số lượng mẫu nhỏ nhất trong các tổ hợp: {min_samples_per_group:,}")

# Đặt target cho mỗi tổ hợp (có thể điều chỉnh)
target_per_group = min(min_samples_per_group, 1000)  # Giới hạn tối đa 1000 mẫu/tổ hợp
print(f"📌 Sẽ lấy {target_per_group:,} mẫu cho mỗi tổ hợp")

# Lấy mẫu cân bằng
balanced_dfs = []
summary_data = []

for language in ['en', 'es']:
    for category in categories:
        for sentiment in [-1, 0, 1]:
            # Lọc dữ liệu cho tổ hợp hiện tại
            mask = (combined_df['language'] == language) & \
                   (combined_df['category'] == category) & \
                   (combined_df['sentiment'] == sentiment)
            group_df = combined_df[mask]
            
            if len(group_df) > 0:
                # Lấy mẫu
                n_samples = min(len(group_df), target_per_group)
                if len(group_df) >= target_per_group:
                    sampled = group_df.sample(n=target_per_group, random_state=42)
                else:
                    sampled = group_df
                
                balanced_dfs.append(sampled)
                
                lang_name = 'EN' if language == 'en' else 'ES'
                sent_name = 'Neg' if sentiment == -1 else ('Neu' if sentiment == 0 else 'Pos')
                summary_data.append({
                    'Language': lang_name,
                    'Category': category,
                    'Sentiment': sent_name,
                    'Original': len(group_df),
                    'Sampled': n_samples
                })

# Tạo summary DataFrame
summary_df = pd.DataFrame(summary_data)
print("\n📋 Tóm tắt quá trình lấy mẫu:")
print(summary_df.to_string(index=False))

# Gộp tất cả các mẫu đã cân bằng
final_df = pd.concat(balanced_dfs, ignore_index=True)

# Trộn ngẫu nhiên
final_df = final_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Sắp xếp lại thứ tự các cột
final_df = final_df[['text', 'sentiment', 'score', 'language', 'category', 'source']]

# ==================================================
# HIỂN THỊ THÔNG TIN CUỐI CÙNG
# ==================================================
print("\n" + "="*80)
print("THÔNG TIN DATASET SAU KHI CÂN BẰNG")
print("="*80)
print(f"Tổng số dòng: {len(final_df):,}")

print(f"\n📋 Các cột: {final_df.columns.tolist()}")

print(f"\n📊 Phân bố Ngôn ngữ (cân bằng):")
for lang, count in final_df['language'].value_counts().items():
    lang_name = 'English' if lang == 'en' else 'Spanish'
    print(f"   {lang_name} ({lang}): {count:,} ({count/len(final_df)*100:.2f}%)")

print(f"\n📊 Phân bố Category (cân bằng):")
for cat, count in final_df['category'].value_counts().items():
    print(f"   {cat}: {count:,} ({count/len(final_df)*100:.2f}%)")

print(f"\n📊 Phân bố Sentiment (cân bằng):")
for sent, count in final_df['sentiment'].value_counts().sort_index().items():
    sent_name = 'Negative' if sent == -1 else ('Neutral' if sent == 0 else 'Positive')
    print(f"   {sent_name} ({sent}): {count:,} ({count/len(final_df)*100:.2f}%)")

print(f"\n📊 Crosstab: Language x Category")
print(pd.crosstab(final_df['language'], final_df['category'], margins=True))

print(f"\n📊 Crosstab: Language x Sentiment")
print(pd.crosstab(final_df['language'], final_df['sentiment'], margins=True))

print(f"\n📊 Crosstab: Category x Sentiment")
print(pd.crosstab(final_df['category'], final_df['sentiment'], margins=True))

print(f"\n📊 Crosstab 3D: Số lượng cho mỗi tổ hợp")
group_final = final_df.groupby(['language', 'category', 'sentiment']).size()
print(group_final.unstack(fill_value=0))

print(f"\n🔍 Mẫu dữ liệu:")
final_df.head(10)

In [ ]:
# Kiểm tra cấu trúc trước khi lưu
print("="*80)
print("KIỂM TRA CHẤT LƯỢNG DỮ LIỆU")
print("="*80)
print(f"Shape: {final_df.shape}")

print(f"\n📊 Kiểu dữ liệu các cột:")
print(final_df.dtypes)

print(f"\n❓ Kiểm tra missing values:")
missing_count = final_df.isnull().sum()
print(missing_count)
if missing_count.sum() == 0:
    print("✅ Không có giá trị thiếu!")

print(f"\n🔄 Kiểm tra duplicate:")
dup_count = final_df.duplicated().sum()
print(f"Số dòng trùng lặp: {dup_count}")

print(f"\n⚖️ Kiểm tra cân bằng - Ngôn ngữ:")
print(final_df['language'].value_counts())

print(f"\n⚖️ Kiểm tra cân bằng - Category:")
print(final_df['category'].value_counts())

print(f"\n⚖️ Kiểm tra cân bằng - Sentiment:")
print(final_df['sentiment'].value_counts().sort_index())

# Kiểm tra cân bằng tổng thể
print(f"\n⚖️ Kiểm tra cân bằng tổng thể (Language x Category x Sentiment):")
balance_check = final_df.groupby(['language', 'category', 'sentiment']).size()
print(f"Số lượng unique values: {balance_check.unique()}")
if len(balance_check.unique()) == 1:
    print(f"✅ Dataset hoàn toàn cân bằng! Mỗi tổ hợp có {balance_check.iloc[0]:,} mẫu")
else:
    print(f"⚠️ Dataset chưa hoàn toàn cân bằng:")
    print(f"   Min: {balance_check.min():,}")
    print(f"   Max: {balance_check.max():,}")
    print(f"   Mean: {balance_check.mean():.2f}")

print(f"\n📋 10 dòng mẫu:")
print(final_df.head(10))

# Lưu dataset vào file CSV
output_file = 'Dataset/balanced_multilingual_multicategory_dataset.csv'
final_df.to_csv(output_file, index=False, encoding='utf-8-sig', sep=',')

print(f"\n" + "="*80)
print("✅ ĐÃ LƯU DATASET THÀNH CÔNG!")
print("="*80)
print(f"📁 File: {output_file}")
print(f"📊 Số lượng dòng: {len(final_df):,}")
print(f"📋 Các cột: {', '.join(final_df.columns)}")
print(f"\n✅ Dataset đã được cân bằng hoàn toàn:")
print(f"   🌍 Ngôn ngữ: English (EN) vs Spanish (ES)")
print(f"   📁 Category: {', '.join(final_df['category'].unique())}")
print(f"   😊 Sentiment: Negative vs Neutral vs Positive")
print(f"\n🎯 Mỗi tổ hợp (Language x Category x Sentiment) có số lượng mẫu bằng nhau")
print("="*80)

# Hiển thị phân bố chi tiết cuối cùng
print(f"\n📊 PHÂN BỐ CHI TIẾT CUỐI CÙNG")
print("="*80)

print("\n1️⃣ Language x Category:")
ct1 = pd.crosstab(final_df['language'], final_df['category'], margins=True)
print(ct1)

print("\n2️⃣ Language x Sentiment:")
ct2 = pd.crosstab(final_df['language'], final_df['sentiment'], margins=True)
print(ct2)

print("\n3️⃣ Category x Sentiment:")
ct3 = pd.crosstab(final_df['category'], final_df['sentiment'], margins=True)
print(ct3)

print("\n4️⃣ Phân bố 3 chiều (Language x Category x Sentiment):")
for lang in final_df['language'].unique():
    lang_name = 'English (EN)' if lang == 'en' else 'Spanish (ES)'
    print(f"\n   {lang_name}:")
    lang_data = final_df[final_df['language'] == lang]
    ct = pd.crosstab(lang_data['category'], lang_data['sentiment'])
    print(ct)